In [11]:
def deco(func):
    def inner():
        func()
        print('running inner()')
    return inner

In [12]:
@deco
def target():
    print('running target()')

In [13]:
target()

running target()
running inner()


In [21]:
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

class A():

    @register
    def f1(self):
        print('running f1()')

    @register
    def f2(self):
        print('running f2()')

    def f3(self):
        print('running f3()')

print(registry)

running register(<function A.f1 at 0x10c7b35e0>)
running register(<function A.f2 at 0x10c7b3310>)
[<function A.f1 at 0x10c7b35e0>, <function A.f2 at 0x10c7b3310>]


In [ ]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

In [3]:
avg_1 = make_averager()
avg_2 = make_averager()

for i in range(10):
    print(avg_1(i), end=', ')
    print(avg_2(i+100))

0.0, 100.0
0.5, 100.5
1.0, 101.0
1.5, 101.5
2.0, 102.0
2.5, 102.5
3.0, 103.0
3.5, 103.5
4.0, 104.0
4.5, 104.5


In [6]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [7]:
avg = make_averager()
avg(10)

10.0

In [10]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [17]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12475991s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000525s] factorial(2) -> 2
[0.00000930s] factorial(3) -> 6
[0.00001216s] factorial(4) -> 24
[0.00001574s] factorial(5) -> 120
[0.00001884s] factorial(6) -> 720
6! = 720


In [16]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

functools.lru_cache 是非常实用的装饰器，它实现了备忘（memoization）功能

In [22]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00004506s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000715s] fibonacci(2) -> 1
[0.00001383s] fibonacci(3) -> 2
[0.00006700s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000620s] fibonacci(2) -> 1
[0.00001287s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000596s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00000787s] fibonacci(2) -> 1
[0.00001502s] fibonacci(3) -> 2
[0.00002789s] fibonacci(4) -> 3
[0.00004697s] fibonacci(5) -> 5
[0.00012016s] fibonacci(6) -> 8
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00000811s] fibonacci(2) -> 1
[0.00001383s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000

55

In [23]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00002503s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00003099s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00003695s] fibonacci(6) -> 8
[0.00000000s] fibonacci(7) -> 13
[0.00004244s] fibonacci(8) -> 21
[0.00000000s] fibonacci(9) -> 34
[0.00004721s] fibonacci(10) -> 55


55

In [ ]:
from functools import singledispatch
from collections import abc

@singledispatch
def test(obj):
    return 'test'

@test.register(str)
def _(text):
    return 'test_str'

@test.register(list)
def _(text):
    return 'test_list'


@test.register(dict)
@test.register(int)
def _(text):
    return 'test_a'

In [27]:
test('a')

'test_str'

In [32]:
import numbers
# 获取类C的父类
parent_classes = numbers.Integral.__mro__

# 打印父类
for parent in parent_classes:
    print(parent.__name__)

Integral
Rational
Real
Complex
Number
object


In [34]:
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x112f57ee0>)
running main()
registry -> [<function f1 at 0x112f57ee0>]
running f1()
